https://yandex.ru/dev/direct/doc/dg/concepts/overview.html

https://yandex.ru/dev/direct/doc/dg-v4/concepts/About.html

https://yandex.ru/dev/id/doc/dg/oauth/concepts/about.html

https://yandex.ru/dev/id/doc/dg/index.html

https://docs.python.org/3/library/json.html

In [1]:
import requests
import json
import pandas as pd

# from flask import Flask, request, jsonify, redirect
# from urllib.parse import urlencode

In [2]:
# параметры приложения
client_id = '2ea405b379c7474e8bf9e1a3bf31ee2a'
client_secret = '86dcb9f2c8664399b553bc98e44da1df'

# я.директ учетная запись 1
login1 = 'sfedyusnin@yandex.ru'
token1 = 'y0_AQAAAABihNGYAAhQ1wAAAADLKvrhX8L69sCOQii3e7pjpQ0tsAU4Ha4'

# я.директ учетная запись 2
login2 = 'test150822-1@yandex.ru' 
token2 = 'y0_AQAAAABj2FshAAhQ1wAAAADLnEO5KjPw9q9tRUuL9DlWVNgRkdHQDEw'

In [305]:
class YandexDirectEcomru:
    def __init__(self, login, token, sandbox=False):
        self.login = login
        self.token = token
        urls = ['https://api.direct.yandex.ru/v4/json/',
                'https://api.direct.yandex.ru/live/v4/json/',
                'https://api.direct.yandex.com/json/v5/']
        
        sandbox_urls = ['https://api-sandbox.direct.yandex.ru/v4/json/',
                        'https://api-sandbox.direct.yandex.ru/live/v4/json/',
                        'https://api-sandbox.direct.yandex.com/json/v5/']
        
        self.head = {"Authorization": 'Bearer'+' '+self.token,
                     "Accept-Language": "ru",
                     "Client-Login": self.login,
                     "Content-Type": "application/json; charset=utf-8"
                    }
        
        if sandbox is True:
            self.urls = sandbox_urls
        else:
            self.urls = urls

    
    def get_campaigns(self, criteries={}):
        """
        Возвращает параметры кампаний, отвечающих заданным критериям.
        Структура описания критериев:
        https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html#input__CampaignsSelectionCriteria
        """
        method = 'campaigns'
        body = {"method": "get",
                "params": {"SelectionCriteria": criteries,
                           "FieldNames": ["Id", "Name"]
                          }
               }
        
        response = requests.post(self.urls[2]+method, headers=self.head, data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text

    def create_new_wordstat_report(self, phrases, regions=None, lim=10):
        """
        Запускает на сервере формирование отчета о статистике поисковых запросов,
        возвращает идентификатор будущего отчета.
        """
        body = {"method": "CreateNewWordstatReport",
                "param": {"Phrases": phrases,
                          "GeoID": regions
                         },
                "locale": "ru",
                "token": self.token
               }
        head = {"Accept-Language": "ru",
                "Content-Length": "204",
                "Content-Type": "application/json; charset=utf-8"}
        
        if len(phrases) <= lim:
            response = requests.post(self.urls[1], headers=head, 
#                                      data=json.dumps(body),
                                     data=json.dumps(body, ensure_ascii=False).encode('utf8')         
                                    )
            if response.status_code == 200:
                return response.json()
            else:
                return response.text
        else:
            print('Превышен лимит фраз в одном запросе')

    def get_wordstat_report_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReportList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
    
    def get_wordstat_report(self, report_id):
        """
        Возвращает отчет о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def delete_wordstat_report(self, report_id):
        """
        Удаляет отчет о статистике поисковых запросов
        """
        body = {"method": "DeleteWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def create_new_forecast(self, phrases, regions=None, currency='RUB', auc_bids='No'):
        """
        Запускает на сервере формирование прогноза показов, кликов и затрат.
        Возможные значения currency: RUB, CHF, EUR, KZT, TRY, UAH, USD, BYN.
        """
        body = {"method": "CreateNewForecast",
                "param": {"Phrases": phrases,
                          "Categories": [],
                          "GeoID": regions,
                          "Currency": currency,
                          "AuctionBids": auc_bids
                         },
                "locale": "ru",
                "token": self.token
               }
        
        head = {"Accept-Language": "ru",
                "Content-Length": "204",
                "Content-Type": "application/json; charset=utf-8"}
        
        response = requests.post(self.urls[1], data=json.dumps(body, ensure_ascii=False).encode('utf8'))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
            
    def get_forecast_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о прогнозируемом количестве
        показов и кликов, затратах на кампанию
        """
        body = {"method": "GetForecastList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def get_forecast(self, forecast_id):
        """
        Возвращает сформированный прогноз показов, кликов и затрат по его идентификатору
        """
        body = {"method": "GetForecast",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text 
      
    def delete_forecast_report(self, forecast_id):
        """
        Удаляет отчет о прогнозируемом количестве показов и кликов, затратах на кампанию
        """
        body = {"method": "DeleteForecastReport",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
 
    @staticmethod
    def create_text_camp_params(s_bid_strat: str,
                                n_bid_strat: str,
                                s_weekly_spend_limit=None,
                                s_bid_ceiling=None,
                                s_goal_id=None,
                                s_average_cpc=None,
                                s_average_cpa=None,
                                s_reserve_return=None,
                                s_roi_coef=None,
                                s_profitability=None,
                                s_crr=None,
                                s_cpa=None,
                                n_limit_percent=100,
                                n_weekly_spend_limit=None,
                                n_bid_ceiling=None,
                                n_goal_id=None,
                                n_average_cpc=None,
                                n_average_cpa=None,
                                n_reserve_return=None,
                                n_roi_coef=None,
                                n_profitability=None,
                                n_crr=None,
                                n_cpa=None,                                
                                settings=[{"Option": "ADD_METRICA_TAG", "Value": "YES"},
                                          {"Option": "ADD_OPENSTAT_TAG", "Value": "NO"},
                                          {"Option": " ADD_TO_FAVORITES ", "Value": "NO"},
                                          {"Option": "ENABLE_AREA_OF_INTEREST_TARGETING", "Value": "YES"},
                                          {"Option": "ENABLE_COMPANY_INFO", "Value": "YES"},
                                          {"Option": "ENABLE_SITE_MONITORING", "Value": "NO"},
                                          {"Option": "EXCLUDE_PAUSED_COMPETING_ADS", "Value": "NO"},
                                          {"Option": "MAINTAIN_NETWORK_CPC", "Value": "NO"},
                                          {"Option": "REQUIRE_SERVICING", "Value": "NO"},
                                          {"Option": "CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED", "Value": "NO"}], 
                                counter_ids=None,
#                                 rel_kw_budget_perc=None,
#                                 rel_kw_opt_goal_id=0,
                                goal_ids=None,
                                goal_vals=None,
                                attr_model="LYDC"
                               ):
        """
        Возвращает словарь с параметрами текстовой кампании
        """
        result = {"TextCampaign":{"BiddingStrategy": {"Search": {},
                                                      "Network": {}}
                                 }}
        
        if settings != None:
            result["TextCampaign"]["Settings"] = settings
            
        if counter_ids != None:
            result["TextCampaign"]["CounterIds"] = {"Items": counter_ids}
            
#         if rel_kw_budget_perc != None:
#             result["TextCampaign"]["RelevantKeywords"] = {"BudgetPercent": rel_kw_budget_perc,
#                                                           "OptimizeGoalId": rel_kw_opt_goal_id}
        if goal_ids != None:
            try:
                goals=[{"GoalId": goal_id, "Value": goal_val*1000000, "IsMetrikaSourceOfValue": "NO"}\
                       for goal_id, goal_val in zip(goal_ids, goal_vals)]
                result["TextCampaign"]["PriorityGoals"] = {"Items": goals}
            except TypeError:
                print('Не корректные параметры ключевых целей, на достижение которых направлена автоматическая корректировка ставок')
         
        if attr_model != None:
            result["TextCampaign"]["AttributionModel"] = attr_model

        try:
            if s_bid_strat == 'HIGHEST_POSITION':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat}
            elif s_bid_strat == 'WB_MAXIMUM_CLICKS':
                if s_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                           "WbMaximumClicks": 
                                                                           {"WeeklySpendLimit": s_weekly_spend_limit*1000000, 
                                                                            "BidCeiling": s_bid_ceiling*1000000}}
                else: 
                    result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                           "WbMaximumClicks": 
                                                                           {"WeeklySpendLimit": s_weekly_spend_limit*1000000}}
            elif s_bid_strat == 'WB_MAXIMUM_CONVERSION_RATE':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "WbMaximumConversionRate": {
                                                                           "WeeklySpendLimit": s_weekly_spend_limit*1000000,
                                                                           "GoalId": s_goal_id}}
                if s_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["WbMaximumConversionRate"].setdefault("BidCeiling", 
                                                                                                              s_bid_ceiling*1000000)

            elif s_bid_strat == 'AVERAGE_CPC':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "AverageCpc": {
                                                                           "AverageCpc": s_average_cpc*1000000}}
                if s_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageCpc"].setdefault("WeeklySpendLimit", 
                                                                                                 s_weekly_spend_limit*1000000)                                                          
            elif s_bid_strat == 'AVERAGE_CPA':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "AverageCpa": {
                                                                           "AverageCpa": s_average_cpa*1000000, 
                                                                           "GoalId": s_goal_id}}
                if s_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageCpa"].setdefault("WeeklySpendLimit", 
                                                                                                 s_weekly_spend_limit*1000000)
                
                if s_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageCpa"].setdefault("BidCeiling",
                                                                                                s_bid_ceiling*1000000)                                                                         
            elif s_bid_strat == 'AVERAGE_ROI':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "AverageRoi": {
                                                                           "ReserveReturn": s_reserve_return,
                                                                           "RoiCoef": s_roi_coef*1000000,
                                                                           "GoalId": s_goal_id}}
                if s_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageRoi"].setdefault("WeeklySpendLimit",
                                                                                                 s_weekly_spend_limit*1000000)
                if s_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageRoi"].setdefault("BidCeiling",
                                                                                                s_bid_ceiling*1000000)
                if s_profitability != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageRoi"].setdefault("Profitability",
                                                                                                s_profitability*1000000)
            elif s_bid_strat == 'AVERAGE_CRR':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "AverageCrr": {
                                                                           "Crr": s_crr,
                                                                            "GoalId": s_goal_id}}
                if s_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["AverageCrr"].setdefault("WeeklySpendLimit",
                                                                                                 s_weekly_spend_limit*1000000)
            elif s_bid_strat == 'PAY_FOR_CONVERSION':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "PayForConversion" : {
                                                                           "Cpa" : s_cpa*1000000,
                                                                           "GoalId" : s_goal_id}}
                if s_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Search"]["PayForConversion"].setdefault("WeeklySpendLimit",
                                                                                                      s_weekly_spend_limit*1000000)
            elif s_bid_strat == 'PAY_FOR_CONVERSION_CRR':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat,
                                                                       "PayForConversionCrr" : {
                                                                           "Crr": s_crr,
                                                                           "GoalId": s_goal_id}}
                if s_weekly_spend_limit != None:
                     result["TextCampaign"]["BiddingStrategy"]["Search"]["PayForConversionCrr"].setdefault("WeeklySpendLimit",
                                                                                                           s_weekly_spend_limit*1000000)
            elif s_bid_strat == 'SERVING_OFF':
                result["TextCampaign"]["BiddingStrategy"]["Search"] = {"BiddingStrategyType": s_bid_strat}
        except TypeError:
            print('Не корректные параметры стратегии показа на поиске')
            return None
        
        try:
            if n_bid_strat == 'NETWORK_DEFAULT':
                if s_bid_strat == 'HIGHEST_POSITION':
                    result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                            "NetworkDefault": {
                                                                                "LimitPercent": n_limit_percent}}
                else:
                    result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                            "NetworkDefault": {}}
            elif n_bid_strat == 'MAXIMUM_COVERAGE':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat}
            elif n_bid_strat == 'WB_MAXIMUM_CLICKS':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "WbMaximumClicks": {
                                                                            "WeeklySpendLimit": n_weekly_spend_limit*1000000}}
                if n_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["WbMaximumClicks"].setdefault("BidCeiling",
                                                                                                      n_bid_ceiling*1000000)
            elif n_bid_strat == 'WB_MAXIMUM_CONVERSION_RATE':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "WbMaximumConversionRate": {
                                                                            "WeeklySpendLimit": n_weekly_spend_limit*1000000,
                                                                            "GoalId": n_goal_id}}
                if n_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["WbMaximumConversionRate"].setdefault("BidCeiling",
                                                                                                      n_bid_ceiling*1000000)
            elif n_bid_strat == 'AVERAGE_CPC':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "AverageCpc": {
                                                                            "AverageCpc": n_average_cpc*1000000}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageCpc"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
            elif n_bid_strat == 'AVERAGE_CPA':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "AverageCpa": {
                                                                            "AverageCpa": n_average_cpa*1000000,
                                                                            "GoalId": n_goal_id}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageCpa"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
                if n_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageCpa"].setdefault("BidCeiling",
                                                                                                  n_bid_ceiling*1000000)
            elif n_bid_strat == 'AVERAGE_ROI':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "AverageRoi": {
                                                                            "ReserveReturn": n_reserve_return,
                                                                            "RoiCoef": n_roi_coef*1000000,
                                                                            "GoalId": n_goal_id}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageRoi"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
                if n_bid_ceiling != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageRoi"].setdefault("BidCeiling",
                                                                                                  n_bid_ceiling*1000000)
                if n_profitability != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageRoi"].setdefault("Profitability",
                                                                                                 n_profitability*1000000)
            elif n_bid_strat == 'AVERAGE_CRR':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "AverageCrr": {
                                                                            "Crr": n_crr,
                                                                            "GoalId": n_goal_id}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["AverageCrr"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
            elif n_bid_strat == 'PAY_FOR_CONVERSION':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "PayForConversion" : {
                                                                            "Cpa": n_cpa*1000000,
                                                                            "GoalId": n_goal_id}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["PayForConversion"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
            elif n_bid_strat == 'PAY_FOR_CONVERSION_CRR':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat,
                                                                        "PayForConversionCrr" : {
                                                                            "Crr": n_crr,
                                                                            "GoalId": n_goal_id}}
                if n_weekly_spend_limit != None:
                    result["TextCampaign"]["BiddingStrategy"]["Network"]["PayForConversionCrr"].setdefault("WeeklySpendLimit",
                                                                                                 n_weekly_spend_limit*1000000)
            elif n_bid_strat == 'SERVING_OFF':
                result["TextCampaign"]["BiddingStrategy"]["Network"] = {"BiddingStrategyType": n_bid_strat}
        except TypeError:
            print('Не корректные параметры стратегии показа в сетях')
            return None
            
        return result
    
    @staticmethod
    def create_campaign(name: str,
                        start_date: str,
                        end_date=None,
                        client_info=None,
                        sms_events=None,
                        sms_time_from="9:00",
                        sms_time_to="21:00",
                        email=None,
                        email_ch_pos_interval=60,
                        email_warning_bal=20,
                        email_send_acc_news="NO",
                        email_send_warnings="NO",
                        timezone="Europe/Moscow",
                        daily_budget_amount=None,
                        daily_budget_mode=None,
                        negative_keywords=None,
                        blocked_ips=None,
                        excluded_sites=None,
                        text_campaign_params=None,
                        mobile_app_campaign_params=None,
                        dynamic_text_campaign_params=None,
                        cpm_banner_campaign_params=None,
                        smart_campaign_params=None,
                        time_targeting_shedule=None,
                        time_targeting_cons_working_weekends=None,
                        time_targeting_suspend_on_holidays=None,
                        time_targeting_bid_percent=None,
                        time_targeting_start_hour=None,
                        time_targeting_end_hour=None
                       ):
        """
        Возвращает словарь с параметрами кампании
        """
        result = {"Name": name,
                 "StartDate": start_date}
        
        if client_info != None:
            result["ClientInfo"] = client_info
            
        if sms_events != None:
            result.setdefault("Notification", {})
            result["Notification"].setdefault("SmsSettings", {})
            result["Notification"]["SmsSettings"] = {"Events": sms_events,
                                                      "TimeFrom": sms_time_from,
                                                      "TimeTo": sms_time_to}   
        if email != None:
            result.setdefault("Notification", {})
            result["Notification"].setdefault("EmailSettings", {})
            result["Notification"]["EmailSettings"] = {"Email": email,
                                                        "CheckPositionInterval": email_ch_pos_interval,
                                                        "WarningBalance": email_warning_bal,
                                                        "SendAccountNews": email_send_acc_news,
                                                        "SendWarnings": email_send_warnings}
            
        if timezone != None:
            result["TimeZone"] = timezone
            
        if daily_budget_amount != None and daily_budget_mode != None:
            result["DailyBudget"] = {"Amount": daily_budget_amount*1000000,
                                     "Mode": daily_budget_mode}
        
        if end_date != None:
            result["EndDate"] = end_date
            
        if negative_keywords != None:
            result["NegativeKeywords"] = {"Items": negative_keywords}
            
        if  blocked_ips != None:
            if len(blocked_ips) <= 25:
                result["BlockedIps"] = {"Items": blocked_ips}
            else:
                print('Количество blocked_ips превышает максимальное значение')
                
        if excluded_sites != None:
            if len(excluded_sites) <= 1000:
                for site in excluded_sites:
                    if len(site) > 255:
                        print('Длина строки excluded_site превышает максимальное значение')
                        return None
                        break
                result["ExcludedSites"] = excluded_sites
            else:
                print('Количество excluded_sites превышает максимальное значение')
        
        if text_campaign_params != None:
            result["TextCampaign"] = text_campaign_params
        elif mobile_app_campaign_params != None:
            result["MobileAppCampaign"] = mobile_app_campaign_params
        elif dynamic_text_campaign_params != None:
            result["DynamicTextCampaign"] = dynamic_text_campaign_params
        elif cpm_banner_campaign_params != None:
            result["CpmBannerCampaign"] = cpm_banner_campaign_params
        elif smart_campaign_params != None:
            result["SmartCampaign"] = smart_campaign_params
        
        if time_targeting_shedule != None:
            result.setdefault("TimeTargeting", {})
            result["TimeTargeting"]["Schedule"] =  {"Items": time_targeting_shedule}
            
        if time_targeting_cons_working_weekends != None:
            result.setdefault("TimeTargeting", {})
            result["TimeTargeting"]["ConsiderWorkingWeekends"] = time_targeting_cons_working_weekends
            
        if time_targeting_suspend_on_holidays != None:
            result.setdefault("TimeTargeting", {})
            if time_targeting_suspend_on_holidays == "Yes":
                result["TimeTargeting"]["HolidaysSchedule"] = {"SuspendOnHolidays": time_targeting_suspend_on_holidays}
            elif time_targeting_suspend_on_holidays == "No":
                result["TimeTargeting"]["HolidaysSchedule"] = {"SuspendOnHolidays": time_targeting_suspend_on_holidays,
                                                               "BidPercent": time_targeting_bid_percent,
                                                               "StartHour": time_targeting_start_hour,
                                                               "EndHour": time_targeting_end_hour} 

        return result

In [306]:
direct = YandexDirectEcomru(login=login2, token=token2, sandbox=True)

# print(direct.get_campaigns.__doc__)
print(direct.get_campaigns())

print(direct.get_wordstat_report_list())
print(direct.get_forecast_list())

{'result': {'Campaigns': [{'Id': 440758, 'Name': 'Test API Sandbox campaign 1'}, {'Id': 440759, 'Name': 'Test API Sandbox campaign 2'}, {'Name': 'Test API Sandbox campaign 3', 'Id': 440760}, {'Id': 441121, 'Name': 'новая кампания'}, {'Name': 'новая кампания 2', 'Id': 441129}, {'Name': 'новая кампания 2', 'Id': 441130}]}}
{'data': []}
{'data': []}


In [307]:
direct.create_campaign(client_info='test150822-1', 
                       sms_events=['MONITORING', 'MONEY_OUT'],
                      email='test150822-1@yandex.ru',
#                        timezone=None,
                       name='тест',
                       start_date='2022-08-30',
                       daily_budget_amount=800,
                       daily_budget_mode='STANDARD',
                       end_date='2022-09-01',
                       negative_keywords=['амазон', 'вайлдберриз'],
                       blocked_ips=['10.0.100.101', '10.0.100.102'],
                       excluded_sites=None,
                       time_targeting_shedule=[1, 2, 3, 4, 5, 6, 7],
                        time_targeting_cons_working_weekends="Yes",
                        time_targeting_suspend_on_holidays="No",
                        time_targeting_bid_percent=None,
                        time_targeting_start_hour=None,
                        time_targeting_end_hour=None
                      )

{'Name': 'тест',
 'StartDate': '2022-08-30',
 'ClientInfo': 'test150822-1',
 'Notification': {'SmsSettings': {'Events': ['MONITORING', 'MONEY_OUT'],
   'TimeFrom': '9:00',
   'TimeTo': '21:00'},
  'EmailSettings': {'Email': 'test150822-1@yandex.ru',
   'CheckPositionInterval': 60,
   'WarningBalance': 20,
   'SendAccountNews': 'NO',
   'SendWarnings': 'NO'}},
 'TimeZone': 'Europe/Moscow',
 'DailyBudget': {'Amount': 800000000, 'Mode': 'STANDARD'},
 'EndDate': '2022-09-01',
 'NegativeKeywords': {'Items': ['амазон', 'вайлдберриз']},
 'BlockedIps': {'Items': ['10.0.100.101', '10.0.100.102']},
 'TimeTargeting': {'Schedule': {'Items': [1, 2, 3, 4, 5, 6, 7]},
  'ConsiderWorkingWeekends': 'Yes',
  'HolidaysSchedule': {'SuspendOnHolidays': 'No',
   'BidPercent': None,
   'StartHour': None,
   'EndHour': None}}}

In [308]:
[str(x+1) for x in range(10)]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [309]:
direct.create_text_camp_params(s_bid_strat='HIGHEST_POSITION',
                               n_bid_strat='NETWORK_DEFAULT')

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'HIGHEST_POSITION'},
   'Network': {'BiddingStrategyType': 'NETWORK_DEFAULT',
    'NetworkDefault': {'LimitPercent': 100}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [310]:
direct.create_text_camp_params(s_bid_strat='WB_MAXIMUM_CLICKS', 
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                              n_bid_strat='NETWORK_DEFAULT')

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
    'WbMaximumClicks': {'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}},
   'Network': {'BiddingStrategyType': 'NETWORK_DEFAULT',
    'NetworkDefault': {}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [311]:
direct.create_text_camp_params(s_bid_strat='WB_MAXIMUM_CLICKS', 
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                              n_bid_strat='MAXIMUM_COVERAGE')

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
    'WbMaximumClicks': {'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}},
   'Network': {'BiddingStrategyType': 'MAXIMUM_COVERAGE'}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [312]:
direct.create_text_camp_params(s_bid_strat='WB_MAXIMUM_CLICKS', 
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                              n_bid_strat='WB_MAXIMUM_CLICKS',
                              n_weekly_spend_limit=5000,
                              n_bid_ceiling=20
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
    'WbMaximumClicks': {'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}},
   'Network': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
    'WbMaximumClicks': {'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [313]:
direct.create_text_camp_params(s_bid_strat='WB_MAXIMUM_CLICKS', 
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                              n_bid_strat='WB_MAXIMUM_CONVERSION_RATE',
                              n_weekly_spend_limit=5000,
                               n_goal_id=13,
                              n_bid_ceiling=20
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
    'WbMaximumClicks': {'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}},
   'Network': {'BiddingStrategyType': 'WB_MAXIMUM_CONVERSION_RATE',
    'WbMaximumConversionRate': {'WeeklySpendLimit': 5000000000,
     'GoalId': 13,
     'BidCeiling': 20000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [314]:
direct.create_text_camp_params(s_bid_strat='WB_MAXIMUM_CONVERSION_RATE', 
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=None,
                               s_goal_id=13,
                              n_bid_strat='AVERAGE_CPC',
                              n_average_cpc=10,
                              n_weekly_spend_limit=5000
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CONVERSION_RATE',
    'WbMaximumConversionRate': {'WeeklySpendLimit': 5000000000, 'GoalId': 13}},
   'Network': {'BiddingStrategyType': 'AVERAGE_CPC',
    'AverageCpc': {'AverageCpc': 10000000, 'WeeklySpendLimit': 5000000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [315]:
direct.create_text_camp_params(s_bid_strat='AVERAGE_CPC',
                               s_average_cpc=10,
                               s_weekly_spend_limit=None,
                              n_bid_strat='AVERAGE_CPA',
                               n_average_cpa=10,
                               n_goal_id=13,
                              n_weekly_spend_limit=5000,
                              n_bid_ceiling=20
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'AVERAGE_CPC',
    'AverageCpc': {'AverageCpc': 10000000}},
   'Network': {'BiddingStrategyType': 'AVERAGE_CPA',
    'AverageCpa': {'AverageCpa': 10000000,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [316]:
direct.create_text_camp_params(s_bid_strat='AVERAGE_CPA',
                               s_average_cpa=10,
                               s_goal_id=13,
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                               n_bid_strat='AVERAGE_ROI',
                              n_reserve_return=20,
                              n_roi_coef=10,
                               n_goal_id=13,
                               n_weekly_spend_limit=5000,
                              n_bid_ceiling=20,
                               n_profitability=60
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'AVERAGE_CPA',
    'AverageCpa': {'AverageCpa': 10000000,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000}},
   'Network': {'BiddingStrategyType': 'AVERAGE_ROI',
    'AverageRoi': {'ReserveReturn': 20,
     'RoiCoef': 10000000,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000,
     'Profitability': 60000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRAS

In [317]:
direct.create_text_camp_params(s_bid_strat='AVERAGE_ROI',
                               s_reserve_return=20,
                               s_roi_coef=10,
                               s_goal_id=13,
                               s_weekly_spend_limit=5000,
                               s_bid_ceiling=20,
                               s_profitability=60,
                              n_bid_strat='AVERAGE_CRR',
                              n_crr=10,
                               n_goal_id=13,
                               n_weekly_spend_limit=5000,
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'AVERAGE_ROI',
    'AverageRoi': {'ReserveReturn': 20,
     'RoiCoef': 10000000,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000,
     'BidCeiling': 20000000,
     'Profitability': 60000000}},
   'Network': {'BiddingStrategyType': 'AVERAGE_CRR',
    'AverageCrr': {'Crr': 10, 'GoalId': 13, 'WeeklySpendLimit': 5000000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'Attribution

In [318]:
direct.create_text_camp_params(s_bid_strat='AVERAGE_CRR',
                               s_crr=10,
                               s_goal_id=13,
                               s_weekly_spend_limit=5000,
                               n_bid_strat='PAY_FOR_CONVERSION',
                               n_cpa=10,
                               n_goal_id=13,
#                                n_weekly_spend_limit=5000
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'AVERAGE_CRR',
    'AverageCrr': {'Crr': 10, 'GoalId': 13, 'WeeklySpendLimit': 5000000000}},
   'Network': {'BiddingStrategyType': 'PAY_FOR_CONVERSION',
    'PayForConversion': {'Cpa': 10000000, 'GoalId': 13}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [321]:
direct.create_text_camp_params(s_bid_strat='PAY_FOR_CONVERSION',
                               s_cpa=10,
                               s_goal_id=13,
                               s_weekly_spend_limit=5000,
                              n_bid_strat='PAY_FOR_CONVERSION_CRR',
                               n_crr=10,
                              n_goal_id=13,
                              n_weekly_spend_limit=5000
                              )

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'PAY_FOR_CONVERSION',
    'PayForConversion': {'Cpa': 10000000,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000}},
   'Network': {'BiddingStrategyType': 'PAY_FOR_CONVERSION_CRR',
    'PayForConversionCrr': {'Crr': 10,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000}}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [323]:
direct.create_text_camp_params(s_bid_strat='SERVING_OFF',
                              n_bid_strat='SERVING_OFF')

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'SERVING_OFF'},
   'Network': {'BiddingStrategyType': 'SERVING_OFF'}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [324]:
direct.create_text_camp_params(s_bid_strat='PAY_FOR_CONVERSION_CRR',
                               s_crr=10,
                               s_goal_id=13,
                               s_weekly_spend_limit=5000,
                              n_bid_strat='SERVING_OFF')

{'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'PAY_FOR_CONVERSION_CRR',
    'PayForConversionCrr': {'Crr': 10,
     'GoalId': 13,
     'WeeklySpendLimit': 5000000000}},
   'Network': {'BiddingStrategyType': 'SERVING_OFF'}},
  'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
   {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
   {'Option': ' ADD_TO_FAVORITES ', 'Value': 'NO'},
   {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
   {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
   {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
   {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
   {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'NO'},
   {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
   {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
  'AttributionModel': 'LYDC'}}

In [6]:
# text_params = direct.create_text_camp_params(s_bid_strat='AVERAGE_ROI',
#                                               s_bid_params=[1, 2, 3, 4, 5, 6],
#                                               n_bid_strat='AVERAGE_ROI',
#                                               n_bid_params=[10, 20, 30, 40, 50, 60]
#                                               )

In [7]:
# text_params

In [8]:
# text_campaigns = direct.compose_text_camp(campaigns=[text_params])

In [9]:
# campaigns = direct.create_campaign(client_info='test150822-1',
#                                    name='новая кампания',
#                                    start_date='2022-08-30',
#                                    text_campaign_params=text_params['TextCampaign'])  

In [10]:
# campaigns

In [11]:
# direct.add_camp(campaigns=[campaigns])

In [13]:
campaigns = {'ClientInfo': 'test150822-1',
             'Name': 'новая кампания 2',
             'StartDate': '2022-08-30',
#              'EndDate': '',
#              'NegativeKeywords': {'Items': ['']},
             'TextCampaign': {'BiddingStrategy': {'Search': {'BiddingStrategyType': 'WB_MAXIMUM_CLICKS',
                                                             'WbMaximumClicks': {"WeeklySpendLimit": 5000*1000000,
                                                                                 "BidCeiling": 20*1000000},
#                                                              'WbMaximumConversionRate': {"WeeklySpendLimit": None, 
#                                                                                          "BidCeiling": '',
#                                                                                          "GoalId": ''},
#                                                              'AverageCpc': '',
#                                                              'AverageCpa': '',
#                                                              'AverageRoi': '',
#                                                              'AverageCrr': '',
#                                                              'PayForConversionCrr': '',
#                                                              'PayForConversion': ''
                                                            },
                                                  'Network': {'BiddingStrategyType': 'NETWORK_DEFAULT',
                                                              'NetworkDefault': {},
#                                                               'WbMaximumClicks': '',
#                                                               'WbMaximumConversionRate': '',
#                                                               'AverageCpc': '',
#                                                               'AverageCpa': '',
#                                                               'AverageRoi': '',
#                                                               'AverageCrr': '',
#                                                               'PayForConversionCrr': '',
#                                                               'PayForConversion': ''
                                                             }
                                                 },
                              'Settings': [{'Option': 'ADD_METRICA_TAG', 'Value': 'YES'},
                                           {'Option': 'ADD_OPENSTAT_TAG', 'Value': 'NO'},
                                           {'Option': 'ADD_TO_FAVORITES', 'Value': 'NO'},
                                           {'Option': 'ENABLE_AREA_OF_INTEREST_TARGETING', 'Value': 'YES'},
                                           {'Option': 'ENABLE_COMPANY_INFO', 'Value': 'YES'},
                                           {'Option': 'ENABLE_SITE_MONITORING', 'Value': 'NO'},
                                           {'Option': 'EXCLUDE_PAUSED_COMPETING_ADS', 'Value': 'NO'},
#                                            {'Option': 'MAINTAIN_NETWORK_CPC', 'Value': 'YES'},
                                           {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
                                           {'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED', 'Value': 'NO'}],
#                               'CounterIds': {'Items': ''},
#                               'RelevantKeywords': '',
#                               'PriorityGoals': {'Items': ''},
                              'AttributionModel': 'LYDC'},
#              'MobileAppCampaign': '',
#              'DynamicTextCampaign': '',
#              'CpmBannerCampaign': '',
#              'SmartCampaign': '',
#              'TimeTargeting': {'Schedule': {'Items': ''},
#                                'ConsiderWorkingWeekends': 'NO',
#                                'HolidaysSchedule': {'SuspendOnHolidays': 'NO',
#                                                     'BidPercent': '',
#                                                     'StartHour': '',
#                                                     'EndHour': ''}
                              }
            

method = 'campaigns'
body = {"method": "add",
        "params": {"Campaigns": [campaigns]}
       }
response = requests.post(direct.urls[2]+method, headers=direct.head, 
                         data=json.dumps(body, ensure_ascii=False).encode('utf8'))
print(response.status_code)
# print(json.dumps(body, ensure_ascii=False).encode('utf8'))
if response.status_code == 200:
    print(response.json())
else:
    print(response.text)

200
{'result': {'AddResults': [{'Id': 441130}]}}


In [106]:
direct.create_new_wordstat_report(phrases=['автоматизация маркетплейсов', 'аналитик продаж на маркетплейсах', \
                                           'интеграция с маркетплейсами', 'копирование карточек озон -товар',\
                                           'копирование карточки товара озон', 'перенос карточек',\
                                           'программа аналитики маркетплейсов'])

{'data': 7063230}

In [6]:
direct.get_wordstat_report_list()

{'data': [{'StatusReport': 'Done', 'ReportID': 401410058},
  {'StatusReport': 'Done', 'ReportID': 401535000}]}

In [7]:
wordstat_report = direct.get_wordstat_report(report_id=401410058)
wordstat_report

{'data': [{'GeoID': [],
   'SearchedAlso': [{'Shows': 1609, 'Phrase': 'tf2 market'},
    {'Phrase': 'тф2 маркет', 'Shows': 828},
    {'Phrase': 'эвотор маркет', 'Shows': 1990},
    {'Shows': 3146, 'Phrase': 'ас маркет'},
    {'Shows': 745, 'Phrase': 'варф маркет'},
    {'Shows': 17577, 'Phrase': 'асу это'},
    {'Phrase': 'кейс маркет', 'Shows': 1106},
    {'Shows': 1043, 'Phrase': 'я маркет интернет магазин'},
    {'Phrase': 'qtx market net', 'Shows': 735},
    {'Phrase': 'ртс маркет электронный магазин', 'Shows': 676},
    {'Shows': 1914, 'Phrase': 'си маркет'},
    {'Shows': 1811, 'Phrase': 'маркет апп'},
    {'Shows': 1140, 'Phrase': 'смарт маркет онлайн'},
    {'Shows': 5321, 'Phrase': 'app market'},
    {'Phrase': 'ртс маркет электронная торговая площадка', 'Shows': 420},
    {'Phrase': 'бьюти маркет', 'Shows': 2305},
    {'Shows': 501, 'Phrase': 'вф маркет'},
    {'Phrase': 'ca market', 'Shows': 747},
    {'Shows': 443, 'Phrase': 'es13 market'},
    {'Phrase': 'ac market', 'Show

In [8]:
pd.DataFrame(wordstat_report['data'])

,GeoID,SearchedAlso,Phrase,SearchedWith
0,[],"[{'Shows': 1609, 'Phrase': 'tf2 market'}, {'Ph...",автоматизация маркетплейсов,"[{'Shows': 246, 'Phrase': 'автоматизация марке..."
1,[],"[{'Phrase': 'маркет гуру', 'Shows': 21997}, {'...",аналитик продаж на маркетплейсах,"[{'Shows': 1249, 'Phrase': 'аналитик продаж на..."
2,[],"[{'Phrase': 'березка агрегатор торговли', 'Sho...",интеграция с маркетплейсами,"[{'Shows': 1309, 'Phrase': 'интеграция с марке..."
3,[],"[{'Shows': 3665, 'Phrase': 'ozon карта отзывы'...",копирование карточек озон -товар,"[{'Shows': 86, 'Phrase': 'копирование карточек..."
4,[],"[{'Shows': 8248, 'Phrase': 'ozon card'}, {'Phr...",копирование карточки товара озон,"[{'Shows': 50, 'Phrase': 'копирование карточки..."
5,[],"[{'Shows': 445140, 'Phrase': 'карта перевода'}...",перенос карточек,"[{'Shows': 934, 'Phrase': 'перенос карточек'},..."
6,[],"[{'Shows': 31496, 'Phrase': 'аналит нет'}, {'P...",программа аналитики маркетплейсов,[{'Phrase': 'программа аналитики маркетплейсов...


In [9]:
# direct.delete_wordstat_report(report_id=7040831)

In [10]:
direct.create_new_forecast(phrases=['автоматизация маркетплейсов', 'аналитик продаж на маркетплейсах', \
                                   'интеграция с маркетплейсами', 'копирование карточек озон -товар',\
                                   'копирование карточки товара озон', 'перенос карточек',\
                                   'программа аналитики маркетплейсов'])

{'data': 401538707}

In [11]:
direct.get_forecast_list()

{'data': [{'ForecastID': 401413197, 'StatusForecast': 'Done'},
  {'ForecastID': 401538707, 'StatusForecast': 'Pending'}]}

In [12]:
forecast = direct.get_forecast(forecast_id=401413197)
forecast

{'data': {'Phrases': [{'Max': 174.62,
    'Shows': 246,
    'PremiumCTR': 11.67,
    'PremiumMin': 378.94,
    'FirstPlaceClicks': 3,
    'Clicks': 3,
    'CTR': 2.8,
    'IsRubric': 'No',
    'Phrase': 'автоматизация маркетплейсов',
    'Currency': 'RUB',
    'PremiumMax': 1304.67,
    'FirstPlaceCTR': 2.68,
    'PremiumClicks': 28,
    'Min': 81.1},
   {'Min': 240.28,
    'Currency': 'RUB',
    'PremiumClicks': 177,
    'FirstPlaceCTR': 1.29,
    'PremiumMax': 2515.35,
    'CTR': 1.15,
    'IsRubric': 'No',
    'Phrase': 'аналитик продаж на маркетплейсах',
    'Clicks': 6,
    'Max': 405.25,
    'PremiumCTR': 15.99,
    'PremiumMin': 531.64,
    'FirstPlaceClicks': 7,
    'Shows': 1249},
   {'Clicks': 9,
    'Shows': 1309,
    'FirstPlaceClicks': 11,
    'PremiumCTR': 10.63,
    'PremiumMin': 387.26,
    'Max': 419.99,
    'PremiumClicks': 137,
    'Currency': 'RUB',
    'FirstPlaceCTR': 2.05,
    'PremiumMax': 1850.99,
    'Min': 185.99,
    'Phrase': 'интеграция с маркетплейсами',


In [14]:
pd.DataFrame(forecast['data']['Phrases'])

,Max,Shows,PremiumCTR,PremiumMin,FirstPlaceClicks,Clicks,CTR,IsRubric,Phrase,Currency,PremiumMax,FirstPlaceCTR,PremiumClicks,Min
0,174.62,246,11.67,378.94,3,3,2.80,No,автоматизация маркетплейсов,RUB,1304.67,2.68,28,81.10
1,405.25,1249,15.99,531.64,7,6,1.15,No,аналитик продаж на маркетплейсах,RUB,2515.35,1.29,177,240.28
2,419.99,1309,10.63,387.26,11,9,1.78,No,интеграция с маркетплейсами,RUB,1850.99,2.05,137,185.99
3,38.67,86,8.00,249.87,2,2,9.52,No,копирование карточек озон -товар,RUB,249.87,9.09,2,17.14
4,17.05,50,16.67,103.26,2,2,28.57,No,копирование карточки товара озон,RUB,235.19,28.57,2,17.05
5,23.91,934,3.61,200.28,2,2,1.55,No,перенос карточек,RUB,425.83,1.48,7,14.16
6,369.45,405,17.88,652.32,4,4,2.70,No,программа аналитики маркетплейсов,RUB,2417.50,2.58,54,206.81


In [21]:
forecast['data']['Common']

{'Geo': '0',
 'Min': 4282.83,
 'PremiumClicks': 407,
 'FirstPlaceClicks': 31,
 'PremiumMin': 195098.72,
 'Shows': 4279,
 'Max': 9617.56,
 'Clicks': 28}

In [22]:
# direct.delete_forecast_report(forecast_id=7037265)

In [23]:
direct.get_forecast_list()

{'data': [{'StatusForecast': 'Done', 'ForecastID': 401413197},
  {'ForecastID': 401538707, 'StatusForecast': 'Done'}]}

In [5]:
# goal_ids = [1, 2, 3, 4, 5]
# goal_vals = [10, 20, 30, 40, 50]
goal_ids = None
goal_vals = None

goals=[{"GoalId": goal_id, "Value": goal_val*1000000, "IsMetrikaSourceOfValue": "NO"} for goal_id, goal_val in zip(goal_ids, goal_vals)]
goals

TypeError: 'NoneType' object is not iterable